In [1]:
import xarray as xr
import numpy as np
from tqdm import tqdm

In [2]:
# Initialize an empty list to store the datasets
datasets = []

variable = 'winds'

years=np.arange(1981,2020)

# Loop through each year and open the dataset
for i, year in tqdm(enumerate(years)):
    storm = xr.open_dataset(f'/home/mduplessis/data/era5/storms/storm_vars/{variable}/storms_{variable}_{year}.nc')
    datasets.append(storm)

39it [00:05,  7.31it/s]


In [4]:
# Concatenate all datasets along a new dimension (e.g., 'time' or 'year')
combined_storms = xr.concat(datasets, dim='time')

In [ ]:
if variable == 'winds':
    # Perform slicing once
    sliced_data = {
        "u10": combined_storms.u10[::2],
        "v10": combined_storms.v10[1::2],
        "time": combined_storms.time[::2],
    }

    # Create the xarray dataset
    ds_storm_wind = xr.Dataset(
        {
            "u10": (["time", "x", "y"], sliced_data["u10"].data),
            "v10": (["time", "x", "y"], sliced_data["v10"].data)
        },
        coords={
            "time": sliced_data["time"].data,
            "x": combined_storms.x.data,
            "y": combined_storms.y.data
        }
    )

    # Update the original dataset if needed
    combined_storms = ds_storm_wind


In [10]:
# Save the concatenated dataset to a new file if needed
combined_storms.to_netcdf(f'/home/mduplessis/data/era5/storms/combined_storms_{variable}.nc')